# Clasificación de sentimientos: Enfoque de Aprendizaje de Máquina

En este notebook, vas a usar [scikit-learn](https://scikit-learn.org/), una de las librerías mas importantes para construcción de modelos de aprendizaje de máquina, para la construcción de un clasificador de sentimientos. El objetivo es identificar si una crítica de una película es positiva o negativa.  Este notebook esta dividido en tres secciones que representan las etapas típicas en la construcción de soluciones de procesamiento de lenguaje natural.

- *Procesamiento del texto*: Vamos a entender el dataset y procesar el texto.
- *Estrategia de representación*: Construiremos una representación básica de bolsa de palabras utilizando la funcionalidad CountVectorizer de sklearn.
- *Modelamiento y evaluación*: Entrenaremos un clasificador Naive Bayes y evaluaremos su desempeño. Tambien utilizaremos el modelo construido para predecir la polaridad en texto construido por nosotros.

In [ ]:
### Lo primero que vamos a realizar es la importación de las librerías necesarias para la construcción de nuestro clasificador

## Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

## Librerias para graficación
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK es una librería particular para PLN. Tiene muchas funcionalidades entre ellas stemming y lista de palabras de parada.
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd


stemmer = nltk.stem.SnowballStemmer('spanish') # Vamos a utlizar el Snowball Stemmer para realizar stemming (nos permite llevar las palabras a una forma estandar).
nltk.download('stopwords') # Lista de palabras de parada en español.

## 1. Procesamiento del texto

#### 1.1. Dataset

Vamos a utlizar una versión modificada del dataset abierto de criticas de películas Españolas de Kaggle: [Críticas películas filmaffinity en Español](https://www.kaggle.com/datasets/ricardomoya/criticas-peliculas-filmaffinity-en-espaniol). El dataset se encuentra en archivo csv "Tutorial-NLP_Analisis_de_Sentimientos.csv".

Para leer este archivo y operarlo vamos a hacer uso de la librería de python [Pandas](https://pandas.pydata.org/) y su método de lectura [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html).

In [ ]:
df = pd.read_csv('Tutorial-NLP_Analisis_de_Sentimientos.csv', sep=',', header=0, index_col= None, engine='python',
                 usecols=['film_name','gender','review_title','review_text','polaridad'])

## Una vez realizado el cargue del dataset en un dataframe de pandas podemos explorarlo!
## Revisemos 5 ejemplos del dataset.
df.sample(5)

Cada ejemplo del dataset (fila) representa una crítica realizada por un usuario a una pelicula.

- La columna '*film_name*' indica el nombre de la película objeto de la crítica.
- La columna '*gender*' indica el género de la película.
- La columna '*review title*' es el título de la crítica.
- La columna '*polaridad*' es la categoría/label asignado a la crítica.


Revisemos la critica en la fila número 2895 en nuestro dataset. Para acceder vamos a utilizar [iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html) que nos permite seleccionar por indice.

In [ ]:
df.loc[2895].review_text

Claramente la crítica es negativa, pero revisemos su categoría/label.

In [ ]:
df.loc[2895].polaridad

Revisemos ahora cuantos ejemplos de críticas negativas y positivas tenemos en nuestro dataset. Esta información es relevante para:
- Seleccionar la relación entre particiones de entrenamiento y testing de nuestro dataset.
- Conocer si las clases estas balanceadas (cada clase tiene el mismo número de ejemplos).
- Seleccionar le modelo de aprendizaje de máquina mas apropiado.

Vamos a utilizar la función [catplot](https://seaborn.pydata.org/generated/seaborn.catplot.html) de la librería seaborn para la elaboración de un histograma.

In [ ]:
sns.catplot(x='polaridad', kind='count', color='r', data=df)
plt.title('Distribución de Ejemplos')
plt.xlabel('Polaridad')
plt.ylabel('Conteo')

Del histograma se puede evidenciar:
- Tenemos 1857 ejemplos de cada clase (positivo, negativo). Las clases estan perfectamente balanceadas.
- En total tenemos 3714 ejemplos en el dataset.

#### 1.2. Unificación de texto

En nuestro dataset tenemos dos fuentes de información textual de la crítica la columna review_text y la columna review_title. Vamos a concatenar ambos textos en una sola nueva columna que denomineros "*texto*". Usaremos su contenido para construir una representación de bolsa de palabras y posteriormente un modelo de Naive Bayes.

In [ ]:
df['texto'] = df['review_title'] + ' ' + df['review_text']
df.head(5)

#### 1.3. Función de procesamiento

Vamos a definir ahora una función "*processing_text*" encargada de modificar el texto de forma apropiada para ser usado en la representación. Es importante aclarar que el procesamiento cambia de acuerdo al dataset y el problema que se esta resolviendo, no existe una única formula mágica.

In [ ]:
'''
* processing_text
* @param texto str
* @return processed_feature str
'''
def processing_text(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()
    # Paso 6: Aplicar stemming. Es una forma de enviar las palabras a una raiz común simplificando de esta manera el vocabulario.
    #         por ejemplo las palabras (absurdo, absurdos) que estan en el review 2895 seran llevados a la raiz común "absurd"
    #         y de esta forma se evita tener dos palabras diferentes con el mismo significado en nuestro vocabulario.
    processed_feature = " ".join([stemmer.stem(i) for i in processed_feature.split()])

	#
 
    return processed_feature



Ahora que ya tenemos nuestra función de procesamiento la vamos a aplicar a nuestro dataset.

In [ ]:
# Primero vamos extraer del dataframe la columna texto y la polaridad y las almacenaremos en las variables
# texto_para_procesar y labels respectivamente
texto_para_procesar = df['texto'].values
labels = df['polaridad'].values

# El texto ya procesado de cada ejemplo en nuestro dataset lo almacenaremos en la variable "texto_procesado"
texto_procesado = []
for sentence in range(0, len(texto_para_procesar)):
    procesado = processing_text(texto_para_procesar[sentence])
    texto_procesado.append(procesado)

In [ ]:
# Comparemos ahora la crítica 2895 procesada vs no procesada.
print("Sin procesar:")
print(texto_para_procesar[2895])
print("---------------------------------")
print("Procesado:")
print(texto_procesado[2895])

.
.
.


Debido al proceso de stemming ahora el texto es mucho mas dificil de leer, sin embargo, el stemming nos permitirá tener un vocabulario reducido.
Ahora solo nos hace falta una etapa del procesamiento que es la eliminación de *palabras de parada*. Las palabras de parada son las palabras más comunes en cualquier idioma:  artículos, preposiciones, pronombres, conjunciones, etc. y no agregan mucha información sobre la semántica del texto. Veamos una lista de estas palabras:

In [ ]:
print(stopwords.words('spanish'))

La eliminación de palabras de parada no siempre es apropiada en las tareas de procesamiento de lenguaje natural. Por ejemplo, si queremos construir un modelo que genere texto coherente es necesario preservar la estructura sintáctica del lenguaje y esto incluye por supuesto las palabras de parada.

## 2. Representación del texto

En esta etapa debemos tomar el texto procesado y representarlo de alguna forma que nos permita operarlo apropiadamente. Vamos a usar la representación más básica que es la *bolsa de palabras*.  Una bolsa de palabras es una representación de texto que describe la ocurrencia de palabras dentro de un documento e implica:

- Un vocabulario de palabras.
- Una ponderación de la presencia de palabras del vocabulario. La forma mas simple es realizar el conteo de las ocurrecias de las palabras del vocabulario en el texto.

Se llama *bolsa de palabras*, porque se descarta cualquier información sobre el orden o la estructura de las palabras en el documento. El modelo solo se preocupa por si las palabras conocidas ocurren en el documento o no. Para construir una bolsa de palabras en sklearn podemos hacer uso de [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) que construye el vocabulario de palabras de nuestro conjunto de datos y transforma el texto en un vector de conteo de palabras.

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=2500, stop_words=stopwords.words('spanish'))
# max_features representa el tamaño del vocabulario. Vamos a permitir 2500 palabras.
# stop_words le indicamos las palabras de parada para que las ignore en el vocabulario.

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray()


#### 2.1. Vocabulario

Revisemos el vocabulario. *CountVectorizer* tomo todas las críticas del dataset, extrajo todas las palabras diferentes y calculo el número de apariciones. De todas las palabras matuvo las 2500 mas frecuentes sin tener en cuenta la lista de palabras de parada.

In [ ]:
# Para ver las primeras 10 palabras del vocabulario podemos hacer uso del método get_feature_names
vectorizer.get_feature_names_out()[0:10]

In [ ]:
texto_features

#### 2.2. Representación vectorial del texto

Una vez construido el vocabulario, *CountVectorizer* construye una representación vectorial de cada crítica. El valor de cada dimensión de dicho vector corresponde al número  de ocurrencias de una palabra del vocabulario. Este vector tendra po lo tanto tantas dimensiones como palabras en el vocabulario, en nuestro caso 2500.

Volvamos al texto procesado de la crítica 2895:

*"no pud acab de verl me aburr no le vi la graci ocho apell vasc much men se la voy ver este desgraci efect secundari el cas es que empec verl con gan ver si por lo men me rei de algun chist per no los distingu no exist en el text ando espesit yo el guion es un absurd sin graci porqu hay absurd gracios per no este si no hay guion no hay peli por much que teng karr a la sard me los imagin los dos compart unas kokotx con xamfain hac confident acerc de lo que tien que hac uno par que le qued una pension decent en realid esta es la critic de las tres cuart part de la pelicul el final me cog hac el colaca par irme la cam
"*

Como se pueden dar cuenta la palabra *absurd* aparece dos veces en el texto. Esta palabra es la número 9 en nuestro vocabulario, por lo tanto la representación vectorial de la crítica 2895 deberá contener un 2 (frecuencia) en la dimension 9.

In [ ]:
len(vectorizer.get_feature_names_out())

In [ ]:
vectorizer.get_feature_names_out()[:10]

In [ ]:
texto_procesado[2895]

In [ ]:
print(len(texto_features[2895])) # Longitud del vector que representa a la crítica 2895.
print(texto_features[2895][0:100]) # 100 primeras posiciones del vector de la crítica. Puede ubicar la dimensión correspondiente a la palabra "absurb"?

## 3. Modelamiento y evaluación

Ahora que tenemos nuestra representación, es necesario construir y evaluar nuestro modelo de Naive Bayes. Necesitamos un subconjunto de los datos para construir nuestro modelo probabilisitico (train) y otro subconjunto para evaluarlo (test).

In [ ]:
# Aca dividimos nuestro dataset en entrenamiento y texto. 20% para evaluar (test) y 80% para entrenamiento (train).
# En las variables X_ quedaran almacenados las presentaciones vectoriales de las críticas.
# En las variables y_ las etiquetas o categorias (polaridad de la crítica).
X_train, X_test, y_train, y_test = train_test_split(texto_features, labels, test_size=0.2, random_state=0)

Ahora vamos a entrenar el modelo. En sklearn [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html?highlight=multinomialnb#sklearn.naive_bayes.MultinomialNB) nos permite entrenar un modelo de Naive Bayes.

In [ ]:
# Ahora entrenemos un modelo simple. Ya conocenos Naive Bayes!!
nb = MultinomialNB()
# El método fit en sklearn permite ejecutar el proceso de entrenamiento.
nb.fit(X_train, y_train)

#### 3.1. Evaluación

Listo, ya tenemos nuesto modelo de Naive Bayes entrenado, es necesario evaluarlo con el cunjunto de test. Vamos a utilizar una metrica conocida como accuracy que representa la relación de críticas para las cuales el modelo predijo correctamente la crítica. Un valor de 1 en el accuracy indica que a todas las criticas se les predijo correctamente la polaridad, mientras que un accuracy de 0 indica que el modelo fallo todas sus predicciones. Existen otras métricas de evaluación pero por ahora lo vamos a mantener simple.

In [ ]:
# Ahora vamos a evaluar que tan bueno es nuestro modelo NB, utlilizando el conjunto de datos de test.
# Para predecir utlizamos el método predict.

predictions = nb.predict(X_test)

In [ ]:
# Ahora calculamos el score de accuracy enviando las predicciónes y los valores reales de polaridad.
print(accuracy_score(y_test, predictions))

Un accuracy mayor al 80% no esta nada mal para un modelo simple como Naive Bayes. Vamos ahora a jugar un poco con el modelo y enviarle texto que nosotros construyamos.

In [ ]:

# Utilizemos la siguiente frase de test (sientanse en la libertad de cambiarla).
test = "Fue la mejor pelicula Pelicula que he visto en mi vida"

# Recuerden el modelo recibe la representación vectorial del texto, no el texto en bruto. Debemos procesar y representar el texto.

test_procesado = processing_text(test) # Aplicamos nuestra función de procesamiento
print("PASO 1 procesamiento:" ,test_procesado)

test_bow =vectorizer.transform([test_procesado]) # Ahora lo representamos como una bolsa de palabras. El vector resultante tiene 2500 posiciones.
print("PASO 2 representación:" ,test_bow) # En esa impresion no aparecen las 2500 posiciones del vector, solo la lista de posiciones que son diferentes de cero.

# Ahora que ya lo tenemos en la representación adecuada lo podemos enviar al modelo para que este prediga su polaridad.
clase_test = nb.predict(test_bow)
print("PASO 3 predecir con el modelo:" ,clase_test)

Modifique el texto en la variable test para que el modelo ahora prediga una polaridad negativa. ¿Se le ocurre alguna forma de confundir al modelo para que la predicción sea incorrecta?, pruebe por ejemplo doble negaciones en el texto.

In [ ]:
## OPCIONAL
## Vamos a profundizar en estos conceptos mas adelante.
cm = confusion_matrix(y_test, predictions,labels=nb.classes_)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nb.classes_)
disp.plot()

In [ ]:
## OPCIONAL
print(classification_report(y_test, predictions, digits=4))